In [1]:
from riotwatcher import LolWatcher, ApiError
from riot_api import key
import pandas as pd
from datetime import datetime

region = 'euw1'

lol = LolWatcher(key)

In [28]:
players = pd.read_csv('players.txt')
matchData = pd.DataFrame({'playerID' : [],'matchID' : [], 'duration (s)' : [], 'start' : [], 'end' : [], 'championName' : [], 'championId' : [], 'pings' : [], 'ePings' : []})

In [29]:
for i in range(players.shape[0]):
    region = players.loc[i,'region']
    pseudo = players.loc[i,'pseudo']
    puuid = lol.summoner.by_name(region,pseudo)['puuid']

    matchs = lol.match.matchlist_by_puuid(region,puuid, count = 5)
    #i = 100
    #while (lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i) != []):
        #matchs.extend(lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i))
        #i = i + 100
    
    for j,i in enumerate(matchs):
        s = matchData.shape[0]
        p = lol.match.by_id(region,i)
        for player in p['info']['participants']:
            if player['puuid'] == puuid:
                cName = player['championName']
                cId = player['championId']
                ePing = player['enemyMissingPings']
                pings = player["allInPings"] + player["assistMePings"] + player["baitPings"] + player["basicPings"] + player["commandPings"] + player["dangerPings"] + player["getBackPings"] + player["holdPings"] + player["needVisionPings"] + player["onMyWayPings"] + player["pushPings"] + player['visionClearedPings'] + ePing + player['enemyVisionPings']
        matchData.loc[j + s,:] = [puuid,p['metadata']['matchId'],p['info']['gameDuration'], p['info']['gameStartTimestamp'], p['info']['gameEndTimestamp'], cName, cId, pings, ePing]


p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIhPjg_7KsMItrmERLTuhXjh5tsLUwwX55w
RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxIaUTwqu4U4zRoEdm_PQ1r2_y5jYkrxvHw


In [30]:
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings
0,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6376297501,2247.0,1.682450e+12,1.682452e+12,Kaisa,145.0,2.0,1.0
2,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6376223722,1846.0,1.682448e+12,1.682449e+12,Tristana,18.0,4.0,1.0
4,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6375983828,1830.0,1.682439e+12,1.682441e+12,MissFortune,21.0,7.0,3.0
6,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6367571487,1857.0,1.681925e+12,1.681927e+12,MissFortune,21.0,4.0,3.0
8,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6365174837,1035.0,1.681760e+12,1.681761e+12,Illaoi,420.0,1.0,0.0
5,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6376297501,2247.0,1.682450e+12,1.682452e+12,Braum,201.0,11.0,1.0
7,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6376223722,1846.0,1.682448e+12,1.682449e+12,Braum,201.0,16.0,3.0
9,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6375983828,1830.0,1.682439e+12,1.682441e+12,Braum,201.0,10.0,1.0
11,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6367571487,1857.0,1.681925e+12,1.681927e+12,Braum,201.0,10.0,0.0
13,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6365910916,1178.0,1.681822e+12,1.681824e+12,Kayn,141.0,0.0,0.0
